In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Klaim.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/sample_submission.csv
/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/Data_Polis.csv


# DATA FOUNDATION

In [2]:
# ============================================================
# STAGE 1 — OPTIMIZED FORECAST VERSION
# Clean • No Leakage • Exposure Aware • Regime Features
# ============================================================

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"

klaim = pd.read_csv(BASE_PATH + "Data_Klaim.csv")
polis = pd.read_csv(BASE_PATH + "Data_Polis.csv")

# ============================================================
# CLEAN COLUMN NAMES
# ============================================================

def clean_columns(df):
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_", regex=False)
        .str.replace("/", "_", regex=False)
        .str.replace("-", "_", regex=False)
    )
    return df

klaim = clean_columns(klaim)
polis = clean_columns(polis)

klaim = klaim.drop_duplicates().reset_index(drop=True)
polis = polis.drop_duplicates().reset_index(drop=True)

# ============================================================
# DATE PARSING
# ============================================================

for col in klaim.columns:
    if "tanggal" in col:
        klaim[col] = pd.to_datetime(klaim[col], errors="coerce")

for col in polis.columns:
    if "tanggal" in col:
        polis[col] = pd.to_datetime(polis[col], errors="coerce")

# ============================================================
# BASIC CLEANING
# ============================================================

klaim = klaim.dropna(subset=["nomor_polis", "tanggal_pasien_masuk_rs"])
klaim["nominal_klaim_yang_disetujui"] = klaim["nominal_klaim_yang_disetujui"].fillna(0)

# Winsorize (robust)
low_q = klaim["nominal_klaim_yang_disetujui"].quantile(0.005)
high_q = klaim["nominal_klaim_yang_disetujui"].quantile(0.995)
klaim["nominal_klaim_yang_disetujui"] = \
    klaim["nominal_klaim_yang_disetujui"].clip(low_q, high_q)

# ============================================================
# MERGE
# ============================================================

df = klaim.merge(polis, on="nomor_polis", how="left")

for col in ["plan_code", "gender", "domisili"]:
    if col in df.columns:
        df[col] = df[col].fillna("UNKNOWN")

# ============================================================
# SERVICE MONTH (NO LEAKAGE)
# ============================================================

df["year_month"] = df["tanggal_pasien_masuk_rs"].dt.to_period("M")

# ============================================================
# EXPOSURE FEATURE (🔥 IMPORTANT)
# ============================================================

exposure_monthly = (
    df.groupby("year_month")
      .agg(active_policies=("nomor_polis","nunique"))
      .reset_index()
)

df = df.merge(exposure_monthly, on="year_month", how="left")

# ============================================================
# DEMOGRAPHIC FEATURES
# ============================================================

if "tanggal_lahir" in df.columns:
    df["age"] = (
        (df["tanggal_pasien_masuk_rs"] - df["tanggal_lahir"]).dt.days / 365
    ).clip(0, 100)
    df["age"] = df["age"].fillna(df["age"].median())

if "tanggal_efektif_polis" in df.columns:
    df["tenure_days"] = (
        df["tanggal_pasien_masuk_rs"] -
        df["tanggal_efektif_polis"]
    ).dt.days.clip(lower=0)
    df["tenure_days"] = df["tenure_days"].fillna(0)

if "tanggal_pasien_keluar_rs" in df.columns:
    df["los"] = (
        df["tanggal_pasien_keluar_rs"] -
        df["tanggal_pasien_masuk_rs"]
    ).dt.days.clip(lower=0)
    df["los"] = df["los"].fillna(0)

# ============================================================
# SEGMENT FEATURES
# ============================================================

df["care_type"] = df["inpatient_outpatient"].astype(str).str.upper().str.strip()
df["care_type"] = df["care_type"].replace(["NAN","NONE"],"UNKNOWN")
df["is_inpatient"] = df["care_type"].eq("IP").astype(int)

rc = df["reimburse_cashless"].astype(str).str.upper().str.strip()
df["is_cashless"] = rc.eq("C").astype(int)

loc = df["lokasi_rs"].astype(str).str.upper().str.strip()

df["rs_bucket"] = np.select(
    [
        loc.eq("INDONESIA"),
        loc.eq("SINGAPORE"),
        loc.eq("MALAYSIA")
    ],
    ["ID","SG","MY"],
    default="OTHER"
)

# ============================================================
# ICD REDUCTION
# ============================================================

df["icd_group_raw"] = (
    df["icd_diagnosis"]
    .astype(str)
    .str.split(".").str[0]
    .str[:3]
)

top_icd = df["icd_group_raw"].value_counts().head(40).index

df["icd_group"] = np.where(
    df["icd_group_raw"].isin(top_icd),
    df["icd_group_raw"],
    "OTHER"
)

# ============================================================
# MONTHLY AGGREGATION (CORE FORECAST TABLE)
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("nomor_polis","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          exposure=("active_policies","first")
      )
      .reset_index()
      .sort_values("year_month")
)

monthly["severity"] = (
    monthly["total_claim"] /
    monthly["frequency"].replace(0,np.nan)
)

# ============================================================
# SAFE TIME-SERIES FEATURES (NO LEAKAGE)
# ============================================================

for col in ["frequency","total_claim","severity"]:
    monthly[f"{col}_lag1"] = monthly[col].shift(1)
    monthly[f"{col}_lag2"] = monthly[col].shift(2)
    monthly[f"{col}_lag3"] = monthly[col].shift(3)

    monthly[f"{col}_roll3"] = monthly[col].shift(1).rolling(3).mean()
    monthly[f"{col}_roll6"] = monthly[col].shift(1).rolling(6).mean()

    monthly[f"{col}_ewm3"] = monthly[col].shift(1).ewm(span=3).mean()
    monthly[f"{col}_ewm6"] = monthly[col].shift(1).ewm(span=6).mean()

    monthly[f"{col}_momentum"] = (
        monthly[f"{col}_lag1"] - monthly[f"{col}_lag2"]
    )

# Volatility regime
monthly["total_vol6"] = (
    monthly["total_claim"].shift(1).rolling(6).std()
)

vol_threshold = monthly["total_vol6"].median()
monthly["high_vol_regime"] = (
    monthly["total_vol6"] > vol_threshold
).astype(int)

# Time index
monthly["month_index"] = np.arange(len(monthly))

# ============================================================
# CLEAN NA (NO BFILL LEAKAGE)
# ============================================================

monthly = monthly.fillna(0)

# ============================================================
# FINAL CHECK
# ============================================================

print("Monthly shape:", monthly.shape)
print("Unique months:", monthly["year_month"].nunique())
print("Missing rate:", monthly.isna().mean().mean())
print("\nSTAGE 1 — OPTIMIZED FORECAST READY")


Monthly shape: (19, 32)
Unique months: 19
Missing rate: 0.0

STAGE 1 — OPTIMIZED FORECAST READY


# TIME-SERIES DATASET ENGINEERING

In [3]:
# ============================================================
# STAGE 2 — ELITE SEGMENT PANEL (FORECAST ALIGNED)
# Compact • Exposure-aware • Regime-aware • No Leakage
# ============================================================

import numpy as np
import pandas as pd

seg_cols = [
    "plan_code",
    "care_type",
    "is_cashless",
    "rs_bucket"
]

# ============================================================
# 1. BUILD SEGMENT MONTHLY
# ============================================================

seg_monthly = (
    df.groupby(["year_month"] + seg_cols)
      .agg(
          frequency=("nomor_polis","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          exposure=("nomor_polis","nunique")
      )
      .reset_index()
      .sort_values(seg_cols + ["year_month"])
      .reset_index(drop=True)
)

# Severity per segmen
seg_monthly["severity"] = (
    seg_monthly["total_claim"] /
    seg_monthly["frequency"].replace(0, np.nan)
)

# ============================================================
# 2. TARGET TRANSFORM
# ============================================================

seg_monthly["log_total"] = np.log1p(seg_monthly["total_claim"])
seg_monthly["log_freq"]  = np.log1p(seg_monthly["frequency"])
seg_monthly["log_sev"]   = np.log1p(seg_monthly["severity"])

# ============================================================
# 3. CALENDAR
# ============================================================

seg_monthly["month"] = seg_monthly["year_month"].dt.month
seg_monthly["month_sin"] = np.sin(2*np.pi*seg_monthly["month"]/12)
seg_monthly["month_cos"] = np.cos(2*np.pi*seg_monthly["month"]/12)

# ============================================================
# 4. CORE LAGS (STRICT NO LEAKAGE)
# ============================================================

seg_monthly = seg_monthly.sort_values(seg_cols + ["year_month"])

for col in ["log_total","log_freq","log_sev"]:

    seg_monthly[f"{col}_lag1"] = \
        seg_monthly.groupby(seg_cols)[col].shift(1)

    seg_monthly[f"{col}_lag2"] = \
        seg_monthly.groupby(seg_cols)[col].shift(2)

    seg_monthly[f"{col}_lag3"] = \
        seg_monthly.groupby(seg_cols)[col].shift(3)

    seg_monthly[f"{col}_roll3"] = \
        seg_monthly.groupby(seg_cols)[col] \
        .transform(lambda x: x.shift(1).rolling(3).mean())

# ============================================================
# 5. MOMENTUM + GROWTH (VERY STRONG SIGNAL)
# ============================================================

seg_monthly["momentum_total"] = (
    seg_monthly["log_total_lag1"] -
    seg_monthly["log_total_lag2"]
)

seg_monthly["momentum_freq"] = (
    seg_monthly["log_freq_lag1"] -
    seg_monthly["log_freq_lag2"]
)

seg_monthly["growth_total"] = (
    seg_monthly["log_total_lag1"] -
    seg_monthly["log_total_lag3"]
)

# ============================================================
# 6. VOLATILITY REGIME (PER SEGMENT)
# ============================================================

seg_monthly["seg_vol3"] = (
    seg_monthly.groupby(seg_cols)["log_total"]
    .transform(lambda x: x.shift(1).rolling(3).std())
)

vol_threshold = seg_monthly["seg_vol3"].median()

seg_monthly["high_vol_regime"] = (
    seg_monthly["seg_vol3"] > vol_threshold
).astype(int)

# ============================================================
# 7. SEGMENT WEIGHT (STABLE AGGREGATION)
# ============================================================

seg_monthly["seg_weight"] = (
    seg_monthly["frequency"] /
    seg_monthly.groupby("year_month")["frequency"].transform("sum")
).fillna(0)

# ============================================================
# 8. DROP EARLY MONTHS (SAFE TRAINING WINDOW)
# ============================================================

seg_model = seg_monthly[
    seg_monthly["log_total_lag3"].notna()
].reset_index(drop=True)

seg_model = seg_model.fillna(0)

# ============================================================
# FINAL CHECK
# ============================================================

print("COMPACT PANEL SHAPE:", seg_model.shape)
print("Unique segments:", seg_model[seg_cols].drop_duplicates().shape[0])
print("Columns:", len(seg_model.columns))
print("\nSTAGE 2 — ELITE SEGMENT PANEL READY")


COMPACT PANEL SHAPE: (414, 33)
Unique segments: 41
Columns: 33

STAGE 2 — ELITE SEGMENT PANEL READY


# MODEL DEVELOPMENT

In [4]:
# ============================================================
# STRUCTURAL v6 (ADAPTIVE SHRINKAGE)
# ============================================================

import numpy as np
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def wmape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    weights = y_true[mask] / y_true[mask].sum()
    return np.sum(weights * np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          exposure=("active_policies","first")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"].replace(0,np.nan)
monthly["claim_rate"] = monthly["frequency"] / monthly["exposure"].replace(0,np.nan)

long_run_sev = monthly["severity"].mean()
long_run_std = monthly["severity"].std()

freq_scores = []
total_scores = []
sev_scores = []

for i in range(8, len(monthly)-1):

    train = monthly.iloc[:i]
    valid = monthly.iloc[i]

    # -------- CLAIM RATE --------
    model_rate = ExponentialSmoothing(
        np.log1p(train["claim_rate"]),
        trend="add",
        damped_trend=True,
        seasonal=None
    ).fit()

    pred_rate = np.expm1(model_rate.forecast(1).iloc[0])
    pred_freq = pred_rate * valid["exposure"]

    # -------- SEVERITY --------
    model_sev = ExponentialSmoothing(
        np.log1p(train["severity"]),
        trend="add",
        damped_trend=True,
        seasonal=None
    ).fit()

    pred_sev_raw = np.expm1(model_sev.forecast(1).iloc[0])

    # -------- ADAPTIVE SHRINKAGE --------
    recent_std = train["severity"].tail(6).std()
    volatility_ratio = recent_std / long_run_std if long_run_std != 0 else 1

    w = 1 / (1 + volatility_ratio)
    w = np.clip(w, 0.6, 0.9)

    pred_sev = w * pred_sev_raw + (1 - w) * long_run_sev

    pred_total = pred_freq * pred_sev

    total_scores.append(wmape([valid["total_claim"]],[pred_total]))
    freq_scores.append(wmape([valid["frequency"]],[pred_freq]))
    sev_scores.append(wmape([valid["severity"]],[pred_sev]))

print("\n==============================")
print("STRUCT v6 WMAPE Frequency :", round(np.mean(freq_scores),2))
print("STRUCT v6 WMAPE Total     :", round(np.mean(total_scores),2))
print("STRUCT v6 WMAPE Severity  :", round(np.mean(sev_scores),2))
print("Estimated Score           :", round(np.mean([
    np.mean(freq_scores),
    np.mean(total_scores),
    np.mean(sev_scores)
]),2))
print("STRUCTURAL v6 COMPLETE")
print("==============================")



STRUCT v6 WMAPE Frequency : 3.97
STRUCT v6 WMAPE Total     : 9.95
STRUCT v6 WMAPE Severity  : 7.25
Estimated Score           : 7.06
STRUCTURAL v6 COMPLETE


# TOTAL CLAIM OPTIMIZATION & VALIDATION, OPTUNA

In [5]:
# ============================================================
# STAGE 4 v17 — DUAL TOTAL ANCHOR MODEL
# AGGRESSIVE 5% ATTEMPT
# ============================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

def wmape(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    mask = y_true != 0
    weights = y_true[mask] / y_true[mask].sum()
    return np.sum(weights * np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

# ============================================================
# BUILD MONTHLY
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          exposure=("active_policies","first")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"].replace(0,np.nan)
monthly["claim_rate"] = monthly["frequency"] / monthly["exposure"].replace(0,np.nan)

monthly["log_rate"] = np.log1p(monthly["claim_rate"])
monthly["log_sev"]  = np.log1p(monthly["severity"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

monthly["t"] = np.arange(len(monthly))

long_run_rate = monthly["claim_rate"].mean()
long_run_sev  = monthly["severity"].mean()

freq_err = []
sev_err = []
total_err = []

for i in range(8, len(monthly)-1):

    train = monthly.iloc[:i]
    valid = monthly.iloc[i]

    # =========================
    # CLAIM RATE MODEL
    # =========================
    X = pd.DataFrame({"const":1, "t":train["t"]})
    model_rate = sm.OLS(train["log_rate"], X).fit()

    future_X = pd.DataFrame({"const":[1], "t":[train["t"].iloc[-1]+1]})
    trend_rate = model_rate.predict(future_X)[0]

    residual_rate = train["log_rate"] - model_rate.predict(X)
    resid_rate = residual_rate.tail(3).mean()

    rate_pred = np.expm1(trend_rate + 0.5*resid_rate)
    rate_pred = 0.9*rate_pred + 0.1*long_run_rate

    freq_pred = rate_pred * valid["exposure"]

    # =========================
    # SEVERITY MODEL
    # =========================
    model_sev = sm.OLS(train["log_sev"], X).fit()
    trend_sev = model_sev.predict(future_X)[0]

    residual_sev = train["log_sev"] - model_sev.predict(X)
    resid_sev = residual_sev.tail(3).mean()

    sev_pred = np.expm1(trend_sev + 0.6*resid_sev)

    recent_std = train["severity"].tail(6).std()
    global_std = monthly["severity"].std()

    vol_ratio = recent_std/global_std if global_std!=0 else 1
    w = np.clip(1/(1+vol_ratio), 0.6, 0.9)

    sev_pred = w*sev_pred + (1-w)*long_run_sev

    # =========================
    # TOTAL ANCHOR MODEL
    # =========================
    model_total = sm.OLS(train["log_total"], X).fit()
    trend_total = model_total.predict(future_X)[0]

    residual_total = train["log_total"] - model_total.predict(X)
    resid_total = residual_total.tail(3).mean()

    total_anchor = np.expm1(trend_total + 0.6*resid_total)

    # =========================
    # STRUCTURAL TOTAL
    # =========================
    total_struct = freq_pred * sev_pred

    # =========================
    # FINAL BLEND
    # =========================
    total_pred = 0.6*total_anchor + 0.4*total_struct

    # anti-drift clamp
    lower = train["total_claim"].tail(6).min()*0.85
    upper = train["total_claim"].tail(6).max()*1.15
    total_pred = np.clip(total_pred, lower, upper)

    # =========================
    # METRICS
    # =========================
    freq_err.append(wmape([valid["frequency"]],[freq_pred]))
    sev_err.append(wmape([valid["severity"]],[sev_pred]))
    total_err.append(wmape([valid["total_claim"]],[total_pred]))

print("\n==============================")
print("FREQ WMAPE :", round(np.mean(freq_err),4))
print("SEV  WMAPE :", round(np.mean(sev_err),4))
print("TOTAL WMAPE:", round(np.mean(total_err),4))
print("EST SCORE  :", round(np.mean([
    np.mean(freq_err),
    np.mean(sev_err),
    np.mean(total_err)
]),4))
print("==============================")



FREQ WMAPE : 0.0414
SEV  WMAPE : 0.0731
TOTAL WMAPE: 0.0974
EST SCORE  : 0.0706


# TEST PREDICTION & KAGGLE SUBMISSION

In [6]:
# ============================================================
# STAGE 5 — FINAL SUBMISSION (CONSISTENT WITH STAGE 4 v17)
# ============================================================

import numpy as np
import pandas as pd
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

BASE_PATH = "/kaggle/input/datasets/dimaspashaakrilian/dsc-itb/"
sample_sub = pd.read_csv(BASE_PATH + "sample_submission.csv")

# ============================================================
# BUILD MONTHLY (IDENTICAL TO STAGE 4 v17)
# ============================================================

monthly = (
    df.groupby("year_month")
      .agg(
          frequency=("claim_id","count"),
          total_claim=("nominal_klaim_yang_disetujui","sum"),
          exposure=("active_policies","first")
      )
      .reset_index()
      .sort_values("year_month")
      .reset_index(drop=True)
)

monthly["severity"] = monthly["total_claim"] / monthly["frequency"].replace(0,np.nan)
monthly["claim_rate"] = monthly["frequency"] / monthly["exposure"].replace(0,np.nan)

monthly["log_rate"] = np.log1p(monthly["claim_rate"])
monthly["log_sev"]  = np.log1p(monthly["severity"])
monthly["log_total"] = np.log1p(monthly["total_claim"])

monthly["t"] = np.arange(len(monthly))

long_run_rate = monthly["claim_rate"].mean()
long_run_sev  = monthly["severity"].mean()

# ============================================================
# PREPARE FUTURE PERIODS
# ============================================================

sample_sub["year"]  = sample_sub["id"].str.split("_").str[0]
sample_sub["month"] = sample_sub["id"].str.split("_").str[1]
sample_sub["month_key"] = sample_sub["year"] + "-" + sample_sub["month"]

future_periods = (
    pd.PeriodIndex(sample_sub["month_key"], freq="M")
      .unique()
      .sort_values()
)

sim_df = monthly.copy()
predictions = {}

# ============================================================
# TRUE RECURSIVE FORECAST (IDENTICAL STRUCTURE)
# ============================================================

for period in future_periods:

    train = sim_df.copy()

    X = pd.DataFrame({"const":1, "t":train["t"]})
    future_t = train["t"].iloc[-1] + 1
    future_X = pd.DataFrame({"const":[1], "t":[future_t]})

    # =========================
    # 1️⃣ CLAIM RATE
    # =========================
    model_rate = sm.OLS(train["log_rate"], X).fit()
    trend_rate = model_rate.predict(future_X)[0]

    residual_rate = train["log_rate"] - model_rate.predict(X)
    resid_rate = residual_rate.tail(3).mean()

    rate_pred = np.expm1(trend_rate + 0.5*resid_rate)
    rate_pred = 0.9*rate_pred + 0.1*long_run_rate

    exposure_next = train["exposure"].iloc[-1]
    freq_pred = rate_pred * exposure_next
    freq_pred = max(freq_pred, 1)

    # =========================
    # 2️⃣ SEVERITY
    # =========================
    model_sev = sm.OLS(train["log_sev"], X).fit()
    trend_sev = model_sev.predict(future_X)[0]

    residual_sev = train["log_sev"] - model_sev.predict(X)
    resid_sev = residual_sev.tail(3).mean()

    sev_pred = np.expm1(trend_sev + 0.6*resid_sev)

    recent_std = train["severity"].tail(6).std()
    global_std = sim_df["severity"].std()

    vol_ratio = recent_std/global_std if global_std!=0 else 1
    w = np.clip(1/(1+vol_ratio), 0.6, 0.9)

    sev_pred = w*sev_pred + (1-w)*long_run_sev

    # =========================
    # 3️⃣ TOTAL ANCHOR
    # =========================
    model_total = sm.OLS(train["log_total"], X).fit()
    trend_total = model_total.predict(future_X)[0]

    residual_total = train["log_total"] - model_total.predict(X)
    resid_total = residual_total.tail(3).mean()

    total_anchor = np.expm1(trend_total + 0.6*resid_total)

    # =========================
    # 4️⃣ STRUCTURAL TOTAL
    # =========================
    total_struct = freq_pred * sev_pred

    # =========================
    # 5️⃣ FINAL BLEND (IDENTICAL TO STAGE 4)
    # =========================
    total_pred = 0.6*total_anchor + 0.4*total_struct

    lower = train["total_claim"].tail(6).min()*0.85
    upper = train["total_claim"].tail(6).max()*1.15
    total_pred = np.clip(total_pred, lower, upper)

    # =========================
    # UPDATE SIMULATION
    # =========================
    new_row = {
        "year_month": period,
        "frequency": freq_pred,
        "total_claim": total_pred,
        "exposure": exposure_next,
        "severity": sev_pred,
        "claim_rate": rate_pred,
        "log_rate": np.log1p(rate_pred),
        "log_sev": np.log1p(sev_pred),
        "log_total": np.log1p(total_pred),
        "t": future_t
    }

    sim_df = pd.concat([sim_df, pd.DataFrame([new_row])], ignore_index=True)

    key = f"{period.year}_{str(period.month).zfill(2)}"
    predictions[f"{key}_Total_Claim"] = total_pred
    predictions[f"{key}_Claim_Frequency"] = freq_pred
    predictions[f"{key}_Claim_Severity"] = sev_pred

# ============================================================
# BUILD SUBMISSION
# ============================================================

submission = sample_sub.copy()
submission["value"] = submission["id"].map(predictions)
submission = submission[["id","value"]]

submission.to_csv("submission.csv", index=False)

print("Submission created — MATCHED TO STAGE 4 v17")
print(submission.head(9))


Submission created — MATCHED TO STAGE 4 v17
                        id         value
0  2025_08_Claim_Frequency  2.623287e+02
1   2025_08_Claim_Severity  5.122518e+07
2      2025_08_Total_Claim  1.254777e+10
3  2025_09_Claim_Frequency  2.649729e+02
4   2025_09_Claim_Severity  5.098988e+07
5      2025_09_Total_Claim  1.264436e+10
6  2025_10_Claim_Frequency  2.636708e+02
7   2025_10_Claim_Severity  5.048497e+07
8      2025_10_Total_Claim  1.263944e+10
